In [1]:
#ignore
from IPython.core.display import HTML,Image
import sys
sys.path.append('/anaconda/')
import config

HTML('<style>{}</style>'.format(config.CSS))

### Introduction

Use of machine learning in the quantitative investment field is, by all indications, skyrocketing.  The proliferation of easily accessible data - both traditional and alternative - along with some very approachable frameworks for machine learning models - is encouraging many to explore the arena.

These financial ML explorers are learning that there are many ways in which using ML to predict financial time series differs greatly from labeling cat pictures or flagging spam.  Application of machine learning to financial time series prediction is made especially difficult due to (1) non-stationarity, (2) low signal-to-noise, and (3) strong feature collinearity within financial data. 

As a consequence, even the most expertly designed ML models will achieve accuracy levels which would seem wholly inadequate in other domains.  It's hard to get excited about an RSQ of 0.10 or a classification accuracy of 0.60, but that is often the reality of well-built models in the domain of asset price prediction.    

In my view, the generic model performance metrics (RSQ, MSE, accuracy, F1, etc...) are not tremendously useful when working in this domain.  Similarly, the traditional quantitative finance metrics (CAGR, sharpe, maxDD, etc...) do not provide as much insight into the models themselves as they do into the particular time period of data used.   

Over the years, I've developed a set of metrics which have proved useful for comparing and optimizing models.  These metrics attempt to measure model performance in terms of _predictive power_ but also in terms of _practicality_, a critically important dimension for those who actually intend to _use_ their models in the real world.    

In this post, I will present a general outline of my approach and will demonstrate a few of the most useful metrics I've added to my standard "scorecard".  I look forward to hearing how others may think to extend the concept.  



### Creating dummy data

I will illustrate the usefulness of this metrics methodology using a simple example of synthetically generated data (see previous posts in this tutorial for explanations of the below method of creating data).  

In [2]:
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like # remove once updated pandas-datareader issue is fixed
# https://github.com/pydata/pandas-datareader/issues/534
import pandas_datareader.data as web
%matplotlib inline

def get_symbols(symbols,data_source, begin_date=None,end_date=None):
    out = pd.DataFrame()
    for symbol in symbols:
        df = web.DataReader(symbol, data_source,begin_date, end_date)[['AdjOpen','AdjHigh','AdjLow','AdjClose','AdjVolume']].reset_index()
        df.columns = ['date','open','high','low','close','volume'] #my convention: always lowercase
        df['symbol'] = symbol # add a new column which contains the symbol so we can keep multiple symbols in the same dataframe
        df = df.set_index(['date','symbol'])
        out = pd.concat([out,df],axis=0) #stacks on top of previously collected data
    return out.sort_index()
        
prices = get_symbols(['AAPL','CSCO','AMZN','YHOO','MSFT'],data_source='quandl',begin_date='2012-01-01',end_date=None)
# note, we're only using real price data to get an accurate date/symbol index set.  

In [3]:
num_obs = prices.close.count()

def add_memory(s,n_days=50,memory_strength=0.1):
    ''' adds autoregressive behavior to series of data'''
    add_ewm = lambda x: (1-memory_strength)*x + memory_strength*x.ewm(n_days).mean()
    out = s.groupby(level='symbol').apply(add_ewm)
    return out

# generate feature data
f01 = pd.Series(np.random.randn(num_obs),index=prices.index)
f01 = add_memory(f01,10,0.1)
f02 = pd.Series(np.random.randn(num_obs),index=prices.index)
f02 = add_memory(f02,10,0.1)
f03 = pd.Series(np.random.randn(num_obs),index=prices.index)
f03 = add_memory(f03,10,0.1)
f04 = pd.Series(np.random.randn(num_obs),index=prices.index)
f04 = f04 # no memory

features = pd.concat([f01,f02,f03,f04],axis=1)

## now, create response variable such that it is related to features
# f01 becomes increasingly important, f02 becomes decreasingly important,
# f03 oscillates in importance, f04 is stationary, finally a noise component is added

outcome =   f01 * np.linspace(0.5,1.5,num_obs) + \
            f02 * np.linspace(1.5,0.5,num_obs) + \
            f03 * pd.Series(np.sin(2*np.pi*np.linspace(0,1,num_obs)*2)+1,index=f03.index) + \
            f04 + \
            np.random.randn(num_obs) * 3 
outcome.name = 'outcome'

## Evaluating Models
Imagine that we created a simple linear model (such as below) and wanted to measure its effectiveness at prediction.  

>Note: we'll follow the walk-forward modeling process described in the [previous post](walk_forward_model_building.html).  If you don't understand the below code snippet (and want to...) please check out that post.

In [4]:
from sklearn.linear_model import LinearRegression

recalc_dates = features.resample('Q',level='date').mean().index.values[:-1]

models = pd.Series(index=recalc_dates)
for date in recalc_dates:
    X_train = features.xs(slice(None,date),level='date',drop_level=False)
    y_train = outcome.xs(slice(None,date),level='date',drop_level=False)
    model = LinearRegression()
    model.fit(X_train,y_train)
    models.loc[date] = model
    
begin_dates = models.index
end_dates = models.index[1:].append(pd.to_datetime(['2099-12-31']))

predictions = pd.Series(index=features.index)

for i,model in enumerate(models): #loop thru each models object in collection
    X = features.xs(slice(begin_dates[i],end_dates[i]),level='date',drop_level=False)
    p = pd.Series(model.predict(X),index=X.index)
    predictions.loc[X.index] = p

So we've got a model, we've got a sizeable set of (out of sample) predictions.  Is the model any good?  Should we junk it, tune it, or trade it?

Since this is a regression model, I'll throw our data into `scikit-learn`'s metrics package.  

In [5]:
import sklearn.metrics as metrics

# make sure we have 1-for-1 mapping between pred and true
common_idx = outcome.dropna().index.intersection(predictions.dropna().index)
y_true = outcome[common_idx]
y_true.name = 'y_true'
y_pred = predictions[common_idx]
y_pred.name = 'y_pred'

standard_metrics = pd.Series()

standard_metrics.loc['explained variance'] = metrics.explained_variance_score(y_true, y_pred)
standard_metrics.loc['MAE'] = metrics.mean_absolute_error(y_true, y_pred)
standard_metrics.loc['MSE'] = metrics.mean_squared_error(y_true, y_pred)
standard_metrics.loc['MedAE'] = metrics.median_absolute_error(y_true, y_pred)
standard_metrics.loc['RSQ'] = metrics.r2_score(y_true, y_pred)

print(standard_metrics)



explained variance    0.254440
MAE                   2.484954
MSE                   9.721411
MedAE                 2.083805
RSQ                   0.254412
dtype: float64


<img src="images/confused_scientist.jpg" width="400">

These stats don't really tell us much by themselves.  You may have an intuition for r-squared so that may give you a level of confidence in the models.  However, even this metric [has problems](https://onlinecourses.science.psu.edu/stat501/node/258/) not to mention does not tell us much about the practicality of this signal from a trading point of view.  

True, we could construct some trading rules around this series of predictions and perform a formal backtest on that.  However, that is quite time consuming and introduces a number of extraneous variables into the equation.  

### A better way...
Below is a method and code framework for evaluating models along several useful dimensions.  Below I'll work through an example of creating a "scorecard" with about a half dozen metrics as a starting point.  

You can feel free to extend this into a longer scorecard which is suited to your needs and beliefs.  In my own trading, I use about 25 metrics in a standard "scorecard" each time I evaluate a model.  You may prefer to use more (or different) metrics but the procedure should be applicable.    

I'll focus only on regression-oriented metrics (i.e., those which use a continuous prediction rather than a binary or classification prediction).  It's trivial to re-purpose the same framework to a classification-oriented environment.  

In this approach, we'll create an extensible _scorecard_ which can contain many custom-defined _metrics_.  These metrics can be combined and adapted in many different ways, some of which I'll lay out in the later part of this example.  

### Preparing our data
However, before implementing specific metrics we need to do some data pre-processing.  It'll become clear why doing this initially will save considerable time later when calculating aggregate metrics.

To create these intermediate values, you'll need the following inputs:
* __y_pred:__ the _continuous variable_ prediction made by your model for each timestep, for each symbol
* __y_true:__ the _continuous variable_ actual outcome for each timestep, for each symbol.  
* __index:__ this is the unique identifier for each prediction or actual result.  If working with a single instrument, then you can simply use date (or time or whatever).  If you're using multiple instruments, a multi-index with (date/symbol) is necessary. 

In other words, if your model is predicting one-day price changes, you'd want your y_pred to be the model's predictions made as of March 9th (for the coming day), indexed as `2017-03-09` and you'd want the actual _future_ outcome which will play out in the next day also aligned to Mar 9th.  This "peeking" convention is very useful for working with large sets of data across different time horizons.  It is described ad nauseum in [this post]().

The raw input data we need to provide might look something like this:

In [6]:
pd.concat([y_pred,y_true],axis=1).tail()

y_pred    y_true
date       symbol                    
2018-03-26 MSFT    0.414205  5.707349
2018-03-27 AAPL   -3.199234 -4.412709
           AMZN    2.648928  0.647332
           CSCO   -1.063083 -1.947601
           MSFT    0.234423  5.756775

We will feed this data into a simple function which will return a dataframe with the y_pred and y_true values, along with several other useful derivative values.  These derivative values include:

* __sign_pred:__ positive or negative sign of prediction
* __sign_true:__ positive or negative sign of true outcome
* __is_correct:__ 1 if sign_pred == sign_true, else 0
* __is_incorrect:__ opposite
* __is_predicted:__ 1 if the model has made a valid prediction, 0 if not.  This is important if models only emit predictions when they have a certain level of confidence
* __result:__ the profit (loss) resulting from betting one unit in the direction of the sign_pred.  This is the continuous variable result of following the model

With this set of intermediate variables already calculated, we can easily calculate the three core metrics of accuracy, edge, and noise as simple one-liners.  For instance: 

In [7]:
def make_df(y_pred,y_true):
    y_pred.name = 'y_pred'
    y_true.name = 'y_true'
    
    df = pd.concat([y_pred,y_true],axis=1)

    df['sign_pred'] = df.y_pred.apply(np.sign)
    df['sign_true'] = df.y_true.apply(np.sign)
    df['is_correct'] = 0
    df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
    df['is_incorrect'] = 0
    df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
    df['is_predicted'] = df.is_correct + df.is_incorrect
    df['result'] = df.sign_pred * df.y_true 
    return df

df = make_df(y_pred,y_train)
df.dropna().tail()

y_pred    y_true  sign_pred  sign_true  is_correct  \
date       symbol                                                         
2017-12-28 MSFT    2.790300  0.269050        1.0        1.0           1   
2017-12-29 AAPL    0.049718 -2.404012        1.0       -1.0           0   
           AMZN    1.472150 -7.034428        1.0       -1.0           0   
           CSCO   -1.194473  0.370690       -1.0        1.0           0   
           MSFT    1.184228  0.217180        1.0        1.0           1   

                   is_incorrect  is_predicted    result  
date       symbol                                        
2017-12-28 MSFT               0             1  0.269050  
2017-12-29 AAPL               1             1 -2.404012  
           AMZN               1             1 -7.034428  
           CSCO               1             1 -0.370690  
           MSFT               0             1  0.217180

### Defining our metrics
The metrics we'll start with here include things like:
* __Accuracy:__ Just as the name suggests, this measures the percent of predictions that were _directionally_ correct vs. incorrect.
* __Edge:__ perhaps the most useful of all metrics, this is the expected value of the prediction over a sufficiently large set of draws.  Think of this like a blackjack card counter who knows the expected profit on each dollar bet when the odds are at a level of favorability
* __Noise:__ critically important but often ignored, the noise metric estimates how dramatically the model's predictions vary from one day to the next.  As you might imagine, a model which abruptly changes its mind every few days is much harder to follow (and much more expensive to follow) than one which is a bit more steady.    

In [8]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    
    return scorecard    

calc_scorecard(df)

accuracy    67.122515
edge         1.445281
noise        2.281503
dtype: float64

Much better.  I now know that we've been directionally correct 68% of the time, and that following this signal would create an edge of 1.5 units per time period.  

Let's keep going.  We can now easily combine and transform things to derive new metrics.  The below function shows several examples, including:
* __y_true_chg__ and __y_pred_chg:__ The average magnitude of change (per period) in y_true and y_pred.  
* __prediction_calibration:__ A simple ratio of the magnitude of our predictions vs. magnitude of truth.  This gives some indication of whether our model is properly tuned to the size of movement in addition to the direction of it.  
* __capture_ratio:__ Ratio of the "edge" we gain by naively following our predictions vs. the actual daily change.  100 would indicate that we were _perfectly_ capturing the true movement of the target variable.


In [9]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    return scorecard    

calc_scorecard(df)

accuracy                  67.122515
edge                       1.445281
noise                      2.281503
y_true_chg                 2.886439
y_pred_chg                 1.600980
prediction_calibration     0.554656
capture_ratio             50.071423
dtype: float64

Additionally, metrics can be easily calculated for only long or short predictions (for a two-sided model) or separately for positions which ended up being winners and losers.  
* __edge_long__ and __edge_short:__ The "edge" for only long signals or for short signals.
* __edge_win__ and __edge_lose:__ The "edge" for only winners or for only losers.

If you've added categorical information to your data (such as industry classification), you can also run these metrics on each category of holdings in your data.


In [10]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    # metrics for a subset of predictions
    scorecard.loc['edge_long'] = df[df.sign_pred == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_short'] = df[df.sign_pred == -1].result.mean()  - df.y_true.mean()

    scorecard.loc['edge_win'] = df[df.is_correct == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_lose'] = df[df.is_incorrect == 1].result.mean()  - df.y_true.mean()

    return scorecard    

calc_scorecard(df)

accuracy                  67.122515
edge                       1.445281
noise                      2.281503
y_true_chg                 2.886439
y_pred_chg                 1.600980
prediction_calibration     0.554656
capture_ratio             50.071423
edge_long                  1.430753
edge_short                 1.427090
edge_win                   3.209777
edge_lose                 -2.206796
dtype: float64

From this toy example, we'd see that the model is predicting with a strong directional accuracy, is capturing about half of the total theoretical profit to be made, makes more on winners than it loses on losers, and is equally valid on both long and short predictions.  If this were real data, I would be rushing to put this model into production!


### Comparing models
The true usefulness of this methodology comes when wanting to make comparisons.  Model A vs Model B.  Last year vs. this year.  Small cap vs. large cap.  

To illustrate, let's say that we're comparing two models, a linear regression vs. a random forest, for performance on a training set and a testing set (pretend for a moment that we didn't adhere to [walk-forward modeling]() practices...).   

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor

X_train,X_test,y_train,y_test = train_test_split(features,outcome,test_size=0.20,shuffle=False)

# linear regression
model1 = LinearRegression().fit(X_train,y_train)
model1_train = pd.Series(model1.predict(X_train),index=X_train.index)
model1_test = pd.Series(model1.predict(X_test),index=X_test.index)

model2 = RandomForestRegressor().fit(X_train,y_train)
model2_train = pd.Series(model2.predict(X_train),index=X_train.index)
model2_test = pd.Series(model2.predict(X_test),index=X_test.index)

# create dataframes for each 
model1_train_df = make_df(model1_train,y_train)
model1_test_df = make_df(model1_test,y_test)
model2_train_df = make_df(model2_train,y_train)
model2_test_df = make_df(model2_test,y_test)

s1 = calc_scorecard(model1_train_df)
s1.name = 'model1_train'
s2 = calc_scorecard(model1_test_df)
s2.name = 'model1_test'
s3 = calc_scorecard(model2_train_df)
s3.name = 'model2_train'
s4 = calc_scorecard(model2_test_df)
s4.name = 'model2_test'

pd.concat([s1,s2,s3,s4],axis=1)

,model1_train,model1_test,model2_train,model2_test
accuracy,68.352152,63.219895,88.823433,59.947644
edge,1.556469,1.148927,2.702832,0.913043
noise,2.182905,2.158154,3.203030,2.492043
y_true_chg,2.909299,2.797717,2.909299,2.797717
y_pred_chg,1.537909,1.527702,2.240762,1.769395
prediction_calibration,0.528618,0.546053,0.770207,0.632443
capture_ratio,53.499797,41.066580,92.903174,32.635274
edge_long,1.524561,1.151913,2.689011,0.925002
edge_short,1.517469,1.168484,2.645668,0.923687
edge_win,3.231354,3.132655,3.123768,3.106294


This quick and dirty scorecard comparison gives us a great deal of useful information.  We learn that:
* The relatively simple linear regression (model1) does a good (unrealistically good...) job of prediction, ciorrect about 68% of the time and capturing about 53% of available price movement (this is very good) during training
* Model1 holds up very well out of sample, performing almost as well on test as train
* Model2, a more complex random forest ensemble model, appears _far_ superior on the training data, capturing 90%+ of available price action, but appears quite overfit and does not perform nearly as well on the test set.


### Summary
In this tutorial, we've covered a framework for evaluating models in a market prediction context  and have demonstrated a few useful metrics.  However, the approach can be extended much further to suit your needs.  You can consider:
* Adding new metrics to the standard scorecard
* Comparing scorecard metrics for subsets of the universe.  For instance, each symbol or grouping of symbols  
* Calculating and plotting performance metrics across time to validate robustness or to identify trends

In the next (and final) [post of this series](ensemble_modeling.html), I'll present a unique framework for creating an _ensemble model_ to blend together the results of your many different forecasting models.  

Please feel free to add to the comment section with your good ideas for useful metrics, with questions/comments on this post, and topic ideas for future posts.  

### One last thing...

If you've found this post useful, please follow [@data2alpha](https://twitter.com/data2alpha) on twitter and forward to a friend or colleague who may also find this topic interesting.

Finally, take a minute to leave a comment below - either to discuss this post or to offer an idea for future posts.  Thanks for reading!